In [ ]:
!pip install --upgrade spacy
!python -m spacy download de_core_news_lg

     |████████████████████████████████| 6.0 MB 20.1 MB/s 
     |████████████████████████████████| 628 kB 24.3 MB/s 
     |████████████████████████████████| 451 kB 35.6 MB/s 
     |████████████████████████████████| 42 kB 785 kB/s 
     |████████████████████████████████| 181 kB 43.0 MB/s 
     |████████████████████████████████| 10.1 MB 40.8 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 572.3

In [ ]:
import spacy
from spacy import displacy
from spacy.matcher import Matcher 
from spacy.tokens import Span
from spacy.util import filter_spans

In [ ]:
nlp = spacy.load('de_core_news_lg')

In [ ]:
nlp = spacy.load('de_core_news_lg')
nlp.add_pipe('merge_entities')
nlp.add_pipe('merge_noun_chunks')

<function spacy.pipeline.functions.merge_noun_chunks>

In [ ]:
ex = 'In der Ostsee zwischen Ystad (Schweden) und Bornholm (Dänemark) ist es zu einem Unfall zwischen zwei Frachtschiffen gekommen, wie mehrere schwedische Medien berichten. Der Unfall hat sich um 3.30 Uhr ereignet. Es handelt sich um das britische Frachtschiff Scot Carrier und das dänische Karin Hoej. Das dänische Frachtschiff soll gekentert sein. Mindestens zwei Menschen seien über Bord gegangen.'

In [ ]:
doc = nlp(ex)

In [ ]:
class TripleExtraction:
  def __init__(self):
    self.subj = ''
    self.pred = ''
    self.obj = ''
    self.aux = False

  def extract(self, sent):
    self.pred = self.extract_pred(sent)
    self.subj = self.extract_subj(sent)
    self.obj = self.extract_obj(sent)

  def extract_subj(self, sent):
    subj = []

    for token in sent:
      if self.aux == True:
        subj.append(' '.join([t.text for t in token.subtree if token.dep_ == 'mo']))
        subj.append(' '.join([t.text for t in token.subtree if token.dep_ == 'sb']))
      else:
        subj.append(' '.join([t.text for t in token.subtree if token.dep_ in ['sb', 'ep']]))
      if len(subj) > 0:
        return ' '.join(subj)

  def extract_pred(self, sent):
    pred = []
    for i, token in enumerate(sent):
      if token.pos_ == 'AUX' and self.aux == False:
        self.aux = True
        rights = [t for t in token.rights]
        pred.append(token.text)
        if doc[i+1].pos_ == 'PRON':
          pred.append(doc[i+1].text)
        if rights:
          pred.append(' '.join([t.text for t in token.rights if t.text not in pred[-1]]))
      elif token.dep_ == 'ROOT':
        pred.append(token.text + ' ' + ' '.join([t.text for t in token.rights]))
    if len(pred) > 0:
      return ' '.join(pred)
  
  def extract_obj(self, sent):
    obj = []
    for token in sent:
      if self.aux == True:
        obj.append(' '.join([t.text for t in token.subtree if token.dep_ == 'oc' and t.text not in obj and t.text not in self.pred]))
      else:
        obj.append(' '.join([t.text for t in token.subtree if token.dep_ in ['mo', 'op'] and t.text not in self.pred]))
    return ' '.join([o for o in obj if len(o) > 0])
        

In [ ]:
for sent in doc.sents:
  tp = TripleExtraction()
  print(sent.text)
  tp.extract(sent)
  print(tp.subj)
  print(tp.pred)
  print(tp.obj)

In der Ostsee zwischen Ystad (Schweden) und Bornholm (Dänemark) ist es zu einem Unfall zwischen zwei Frachtschiffen gekommen, wie mehrere schwedische Medien berichten.
In der Ostsee zwischen Ystad ( Schweden ) und Bornholm ( Dänemark ) 
ist es gekommen .
zu einem Unfall zwischen zwei Frachtschiffen , wie mehrere schwedische Medien berichten
Der Unfall hat sich um 3.30 Uhr ereignet.
 Der Unfall
hat ereignet .
sich um 3.30 Uhr
Es handelt sich um das britische Frachtschiff Scot Carrier und das dänische Karin Hoej.
Es
handelt sich um
das britische Frachtschiff Scot Carrier und das dänische Karin Hoej.
Das dänische Frachtschiff soll gekentert sein.
 Das dänische Frachtschiff
soll sein .
gekentert
Mindestens zwei Menschen seien über Bord gegangen.
 Mindestens zwei Menschen
seien gegangen .
über Bord


In [ ]:
ex2 = 'Das dänische Frachtschiff soll gekentert sein.'
doc = nlp(ex2)

In [ ]:
for token in doc:
  print(token.text, token.pos_, token.dep_)

Das dänische Frachtschiff NOUN sb
soll AUX ROOT
gekentert VERB oc
sein AUX oc
. PUNCT punct


In [ ]:
displacy.render(doc, style='dep', jupyter=True)